In [2]:
fege

NameError: name 'fege' is not defined

In [1]:
import sys
sys.path.append('../_amigocloud')

from sqlalchemy import create_engine, text
from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json
import pandas as pd

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [2]:
sys.path.append('..')

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [4]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [5]:
ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

In [6]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            
            try:
                convert(ruta_docx, ruta_pdf)
                print(ruta_docx)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def cargar_link_to_db(url, cite):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.resumen_control_bio SET link = :url WHERE cite = :cite
            """)
            conn.execute(query, {"url": url, "cite": cite})
            print(f"✔️ Se registro link para CITE: {cite}" )
    except Exception as e:
        print(f"❌ Error al actualizar CITE: {cite}, {e}")

def get_registros_sin_enviar():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_control_bio.resumen_control_bio
            WHERE enviado = false
        """
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error al consultar unidades sin labor: {e}")
        return pd.DataFrame()
    return None

In [7]:
convertir_docx_to_pdf()

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\2626_SLCB_18-09-2025_CONDORI QUEVEDO JORGE_CB-2025-86.docx
✅ Convertido: 2626_SLCB_18-09-2025_CONDORI QUEVEDO JORGE_CB-2025-86.docx
🗑️ Eliminado original: 2626_SLCB_18-09-2025_CONDORI QUEVEDO JORGE_CB-2025-86.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\3412_SLCB_18-09-2025_ESPINOZA VILLCA DIONICIO_CB-2025-82.docx
✅ Convertido: 3412_SLCB_18-09-2025_ESPINOZA VILLCA DIONICIO_CB-2025-82.docx
🗑️ Eliminado original: 3412_SLCB_18-09-2025_ESPINOZA VILLCA DIONICIO_CB-2025-82.docx


  0%|          | 0/1 [00:00<?, ?it/s]

...

C:\Users\Usuario\Desktop\PROYECTOS\_informes\41944_SLCB_18-09-2025_CONDORI QUEVEDO RONALD_CB-2025-83.docx
✅ Convertido: 41944_SLCB_18-09-2025_CONDORI QUEVEDO RONALD_CB-2025-83.docx
🗑️ Eliminado original: 41944_SLCB_18-09-2025_CONDORI QUEVEDO RONALD_CB-2025-83.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\42455_SLCB_16-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-84.docx
✅ Convertido: 42455_SLCB_16-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-84.docx
🗑️ Eliminado original: 42455_SLCB_16-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-84.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\42455_SLCB_17-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-85.docx
✅ Convertido: 42455_SLCB_17-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-85.docx
🗑️ Eliminado original: 42455_SLCB_17-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-85.docx


In [8]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [9]:
contenido = os.listdir(directorio_destino)
print(len(contenido))
contenido

7


['2626_SLCB_18-09-2025_CONDORI QUEVEDO JORGE_CB-2025-86.pdf',
 '3412_SLCB_18-09-2025_ESPINOZA VILLCA DIONICIO_CB-2025-82.pdf',
 '41594_SLCB_18-09-2025_AGROPECUARIA CAMPO DULCE S.R.L._CB-2025-87.pdf',
 '41944_SLCB_18-09-2025_CONDORI QUEVEDO RONALD_CB-2025-81.pdf',
 '41944_SLCB_18-09-2025_CONDORI QUEVEDO RONALD_CB-2025-83.pdf',
 '42455_SLCB_16-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-84.pdf',
 '42455_SLCB_17-09-2025_CIA IND. COMERCIAL HNOS. VICENTE S.R.L._CB-2025-85.pdf']

In [10]:
for file in contenido:
    partes = file.split('_')
    
    cite = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file
    id_drive = '1sEfhWGATb7tkKgr6gcknH3ZA_ed_hGTq'
    path_file = os.path.join(directorio_destino, file)
    print(cite)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    cargar_link_to_db(url_drive, cite)
    if os.path.exists(path_file):
        os.remove(path_file)
        print("✅ Archivo eliminado.")
    else:
        print("⚠️ El archivo no existe.")

CB-2025-86
[INFO] El archivo '2626_SLCB_18-09-2025_CONDORI QUEVEDO JORGE_CB-2025-86.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '2626_SLCB_18-09-2025_CONDORI QUEVEDO JORGE_CB-2025-86.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-86
✅ Archivo eliminado.
CB-2025-82
[INFO] El archivo '3412_SLCB_18-09-2025_ESPINOZA VILLCA DIONICIO_CB-2025-82.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '3412_SLCB_18-09-2025_ESPINOZA VILLCA DIONICIO_CB-2025-82.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-82
✅ Archivo eliminado.
CB-2025-87
[INFO] El archivo '41594_SLCB_18-09-2025_AGROPECUARIA CAMPO DULCE S.R.L._CB-2025-87.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '41594_SLCB_18-09-2025_AGROPECUARIA CAMPO DULCE S.R.L._CB-2025-87.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-87
✅ Archivo eliminado.
CB-2025-81
[INFO] El archivo '41944_SLCB_18-09-2025_CONDORI QUEVEDO RONALD_CB-2025-81.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '4

In [11]:
import psycopg2

In [12]:
def obtener_conexion():
    return psycopg2.connect(
        host='localhost', 
        port=5433, 
        database='utea', 
        user='postgres', 
        password='A123456*')
    
def insertar_mensaje_whatsapp(cod_canero, nombre_canero, numero_contac, mensaje, enviado=False, fecha_enviado=None):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            INSERT INTO notificaciones_wsp.msj_whatsapp (
                cod_canero, nombre_canero, numero_cel, mensaje, enviado, fecha_enviado, numero_contac
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (cod_canero, nombre_canero, 0, mensaje, enviado, fecha_enviado, numero_contac))
        
        conexion.commit()
        cursor.close()
        print("Mensaje insertado correctamente.")
    except Exception as e:
        conexion.rollback()
        print("Error inesperado")
    return None

def marcar_resumen_lib_enviado(id_resumen):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            UPDATE drones_control_bio.resumen_control_bio
            SET enviado = true
            WHERE id = %s
        """, (id_resumen,))
        conexion.commit()
        cursor.close()
        print(f"Mensaje con id={id_resumen} marcado como enviado.")
    except Exception as e:
        conexion.rollback()
        print(f"Error inesperado: {e}")
    finally:
        if conexion:
            conexion.close()
    return None

In [28]:
PATH_XLSX_CONTAC=r'G:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - EQUIPO AVIACION UTEA\\Pulverizacion\\Data\\CONTACTOS.xlsx'

In [29]:
sin_enviar = get_registros_sin_enviar()

In [30]:
cods_ca = list(set(sin_enviar['id']))
cods_ca

[89, 91, 92]

In [31]:
df_contac = pd.read_excel(PATH_XLSX_CONTAC)
df_contac.fillna(0, inplace=True)
df_contac = df_contac[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]

In [32]:
df_contac.head(3)

,cod_ca,nom_ca,telf01,telf02,telf03
0,0,CAÑERO TEST,78194371,0.0,0.0
1,86,AGUILERA TARADELLES JOSE LUIS,67759316,77010520.0,68848905.0
2,180,ELISEO AMURRIO,67559958,0.0,0.0


In [33]:
sin_enviar.head(3)

,id,fecha,institucion,cod_canero,nombre_canero,hectareas,cite,cod_propiedad,propiedad,producto,link,agrocittca,enviado
0,89,2025-09-18,50,3412,ESPINOZA VILLCA DIONICIO,22.30,CB-2025-82,1561,OKINAWA I--DIONISIO ESPINOZA,['TRICHOGRAMMA'],https://drive.google.com/file/d/1gDmLPzM6rW9Ua...,False,False
1,91,2025-09-16,69,42455,CIA IND. COMERCIAL HNOS. VICENTE S.R.L.,146.54,CB-2025-84,2075,PROPIEDAD CERRITO--VICENTE J ENRIQUE,['TRICHOGRAMMA'],https://drive.google.com/file/d/1SshmfpsgrSMcl...,False,False
2,92,2025-09-17,69,42455,CIA IND. COMERCIAL HNOS. VICENTE S.R.L.,253.43,CB-2025-85,2158,EL PROGRESO--VICENTE J ENRIQUE,['TRICHOGRAMMA'],https://drive.google.com/file/d/15Zq16nulIGhUk...,False,False


In [34]:
for index, row in sin_enviar.iterrows():    
    contac_filtro = df_contac[df_contac['cod_ca']==row['cod_canero']]
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        if (len(telefonos_validos) == 0):
            print(f'Codigo: {cod} no tinene numeros validos registrados')
            continue
            
        for num in telefonos_validos:
            msj = f"Saludos *{row['nombre_canero']}*, le escribimos desde *UTEA - GUABIRA* para hacer llegar una copia del informe de servicio de control biologico en caña de azucar. Adjunto informe: {row['link']}"
            insertar_mensaje_whatsapp(
                cod_canero=row['cod_canero'],
                nombre_canero=row['nombre_canero'],
                numero_contac=f'591{num}@s.whatsapp.net',
                mensaje=msj
            )
            marcar_resumen_lib_enviado(row['id'])
    else:
        print(f"Codigo: {row['cod_canero']} - {row['nombre_canero']} no tiene registro telefonico")

Codigo: 3412 - ESPINOZA VILLCA DIONICIO no tiene registro telefonico
Mensaje insertado correctamente.
Mensaje con id=91 marcado como enviado.
Mensaje insertado correctamente.
Mensaje con id=92 marcado como enviado.
